# 2.0 Ingeniería de Características y Preparación de Datos

In [2]:
from pathlib import Path
import sys

current_working_directory = Path.cwd()
project_root = current_working_directory.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [3]:
# Bibliotecas necesarias
import pandas as pd
import os

from src.features.build_features import make_features

### Carga de Datos Preprocesados

In [4]:
processed_data_path = '../data/processed/resampled_taxi_data.parquet'

if not os.path.exists(processed_data_path):
    print(f"Error: No se encontró el archivo de datos procesados en {processed_data_path}.")
    print("Por favor, ejecuta el notebook '1.0-exploracion-inicial-demanda.ipynb' primero para generar este archivo.")
    df_resampled = None
else:
    df_resampled = pd.read_parquet(processed_data_path)
    print(f"Datos cargados desde: {processed_data_path}")
    print(df_resampled.head())

Datos cargados desde: ../data/processed/resampled_taxi_data.parquet
                     num_orders
datetime                       
2018-03-01 00:00:00         124
2018-03-01 01:00:00          85
2018-03-01 02:00:00          71
2018-03-01 03:00:00          66
2018-03-01 04:00:00          43


### Creación de Características Temporales y Lags

Basándonos en la estacionalidad diaria y semanal observada, crearemos las siguientes características:
* **Componentes de fecha y hora:** Año, mes, día, día de la semana, hora.
* **Lags:** Valores pasados del número de pedidos, que son cruciales para las series de tiempo. Un `max_lag` de 24 capturará el patrón del día anterior.
* **Media móvil:** Una media móvil del número de pedidos en un período anterior, para suavizar la tendencia y capturar el comportamiento reciente.

In [5]:
# Parámetros para la creación de características
## Como existe cierto nivel de estacionalidad diaria el max_lag que se utilizará será de 24
## y una media movil de 24 tambien, para que suavice la tendencia diaria.
MAX_LAG = 24
ROLLING_MEAN_SIZE = 24

In [6]:
if df_resampled is not None:
    # Preparando características
    df_features = make_features(df_resampled.copy(), MAX_LAG, ROLLING_MEAN_SIZE)
    print(f'Tamaño del DataFrame con características: {df_features.shape}')
    print(df_features.head())

    features_data_path = '../data/processed/features_taxi_data.parquet'
    df_features.to_parquet(features_data_path, index=True)
    print(f"\nDataFrame con características guardado en: {features_data_path}")
else:
    df_features = None
    print("No se pudo crear el DataFrame de características porque los datos procesados no se cargaron.")


Tamaño del DataFrame con características: (4416, 31)
                     num_orders  year  month  day  dayofweek  hour  lag_1  \
datetime                                                                    
2018-03-01 00:00:00         124  2018      3    1          3     0    NaN   
2018-03-01 01:00:00          85  2018      3    1          3     1  124.0   
2018-03-01 02:00:00          71  2018      3    1          3     2   85.0   
2018-03-01 03:00:00          66  2018      3    1          3     3   71.0   
2018-03-01 04:00:00          43  2018      3    1          3     4   66.0   

                     lag_2  lag_3  lag_4  ...  lag_16  lag_17  lag_18  lag_19  \
datetime                                  ...                                   
2018-03-01 00:00:00    NaN    NaN    NaN  ...     NaN     NaN     NaN     NaN   
2018-03-01 01:00:00    NaN    NaN    NaN  ...     NaN     NaN     NaN     NaN   
2018-03-01 02:00:00  124.0    NaN    NaN  ...     NaN     NaN     NaN     NaN   
20

## Conclusión de Ingeniería de Características

En este notebook, transformamos nuestro DataFrame remuestreado en un conjunto de datos listo para el modelado, agregando características temporales y variables de serie de tiempo que son esenciales para la predicción de la demanda de taxis. Los datos han sido guardados en `data/processed/features_taxi_data.parquet`.